In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Getting Started: Scalar Advection

This tutorial demonstrates the usage of the **Cubed-Sphere Advection Solver**. We simulate the transport of a passive scalar (a Gaussian hill) around the sphere using the high-order Discontinuous Galerkin (DG) method.

**Key Learning Objectives:**
* Understanding the solver configuration (Polynomial order, CFL).
* Running a basic simulation loop.
* Visualizing the Discontinuous Galerkin field on one of the cube faces.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereAdvectionSolver, AdvectionConfig

> **Feature Highlight: Dynamic DG CFL Time-Stepping**
>
> We are proud to introduce our **Dynamic DG CFL Time-Stepping** engine. You no longer need to guess a stable `dt` or manually calculate the CFL condition. The solver automatically analyzes the grid structure, polynomial order ($N$), and the instantaneous flow field to compute the optimal stable time step for every iteration.


In [ ]:
# 1. Configure the Solver
# High-order DG methods need specific resolution settings.
N = 16          # Polynomial Order. Higher N = better spectral accuracy, but stricter CFL.
target_cfl = 0.5 # Safety factor for the CFL condition (0.5 is conservative and robust).

# Configuration Dictionary
# We specify the resolution and backend, but we let the solver handle 'dt'.
config = AdvectionConfig(
    N=N, 
    CFL=target_cfl, 
    backend='numpy' # 'numpy' for compatibility, 'jax' for speed
    # dt is auto-computed by the Dynamic DG CFL engine
)

solver = CubedSphereAdvectionSolver(config)


In [ ]:
# 2. Initialize State (Gaussian Hill)
def gaussian_hill(lon, lat):
    # Center at (0, 0)
    r = np.sqrt(lon**2 + lat**2)
    return np.exp( - (r / 0.5)**2 )

u0 = solver.get_initial_condition(type="custom", func=gaussian_hill)
print(f"Initial state shape: {u0.shape}")

In [ ]:
# 3. Run Simulation
t_final = 1.0
state = u0.copy()
t = 0.0

# Pre-compute the stable time step for the initial state.
# In a full non-linear simulation (like SWE), this would be re-computed every step.
# For constant-velocity advection, one computation is sufficient.
dt = solver.compute_safe_dt(state, cfl=target_cfl)
print(f"Dynamic DG CFL Engine computed dt: {dt:.6f}")

steps = int(t_final / dt)
print(f"Running for {steps} steps...")

for i in range(steps):
    state = solver.step(t, state, dt)
    t += dt 
    if (i+1) % 50 == 0:
        print(f"Step {i+1}/{steps} complete")


In [ ]:
# 4. Visualize Results
# We plot the field on Face 0 (Equatorial Face)
plt.figure(figsize=(8, 6))
plt.imshow(state[0, 0, :, :], origin='lower', extent=[-1, 1, -1, 1])
plt.colorbar(label='Tracer Concentration')
plt.title(f"Advection Result at t={t:.2f} (Face 0)")
plt.show()